# Tarea 2
### Luis Morera Claramunt

A continuación se va a crear un generador de transacciones, se va a utilizar la base de datos original brindada como ejemplo de clase

Primero se inicia sesion y se conecta a la base de datos 'transactions'

In [105]:
import findspark
findspark.init('/Users/Luis/Desktop/spark/spark-2.4.4-bin-hadoop2.7')

from datetime import datetime
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, date_format, udf 
from pyspark.sql.types import DateType

spark = SparkSession \
    .builder \
    .appName("Basic JDBC pipeline") \
    .config("spark.jars", "./postgresql-42.2.9.jar") \
    .getOrCreate()

# Reading single DataFrame in Spark by retrieving all rows from a DB table.
df = spark \
    .read \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/transactions") \
    .option("user", "postgres") \
    .option("password", "password") \
    .option("dbtable", "transactions") \
    .load() 
    
df.show()

+------+-----------+---+-------------------+
|amount|customer_id| id|       purchased_at|
+------+-----------+---+-------------------+
|    55|          1|  1|2017-03-01 09:00:00|
|   125|          1|  2|2017-03-01 10:00:00|
|    32|          1|  3|2017-03-02 13:00:00|
|    64|          1|  4|2017-03-02 15:00:00|
|   128|          1|  5|2017-03-03 10:00:00|
|   333|          2|  6|2017-03-01 09:00:00|
|   334|          2|  7|2017-03-01 09:01:00|
|   333|          2|  8|2017-03-01 09:02:00|
|    11|          2|  9|2017-03-03 20:00:00|
|    44|          2| 10|2017-03-03 20:15:00|
| 17147|        445| 11|2015-03-30 04:52:06|
+------+-----------+---+-------------------+



Se importan las librerias necesarias para calcular las nuevas transacciones

In [106]:
import numpy as np
import pandas as pd
import random
from datetime import datetime
import time

Se crea una función para generar la transacción aleatoria.
La función toma como datos de entrada el máximo valor de compra, máximo id de comprador, fecha de inicio y fecha final, dentro de los cuales se van a generar los datos aleatorios.

In [107]:
# Funcion para generar una transaccion aleatoria
def generateTransaction(max_amount, max_id, start, end):
    
    # Primero se calculan los montos comprados
    amount = np.random.randint(max_amount)
    
    # Se calculan los ids de los compradores
    customer_id = np.random.randint(max_id)
    
    # Se calcula la hora y dia de la compra como string y se hace convierte a tipo datetime
    purchased_at_str = randomDate(start, end, random.random())
    purchased_at = datetime.strptime(purchased_at_str, '%Y-%m-%d %H:%M:%S')
    
    # Creando el dataframe en pandas
    data = [[amount, customer_id, purchased_at]]
    pdf = pd.DataFrame(data, columns = ['amount', 'customer_id', 'purchased_at'])
    
    # Realizando la conversion de pandas a Spark
    df = spark.createDataFrame(pdf)
    
    return df

Se crea una función auxiliar para generar una fecha y tiempo aleatoria, la cual devuelve un objeto tipo string.

In [108]:
# Funcion para calcular una fecha y hora aleatoria
def strTimeProp(start, end, format, prop):

    stime = time.mktime(time.strptime(start, format))
    etime = time.mktime(time.strptime(end, format))

    ptime = stime + prop * (etime - stime)

    return time.strftime(format, time.localtime(ptime))

def randomDate(start, end, prop):
    return strTimeProp(start, end, '%Y-%m-%d %H:%M:%S', prop)

print(randomDate("2008-1-1 1:30:00", "2009-1-1 4:50:00", random.random()))

2008-07-01 18:53:48


Se llama la función para generar la nueva transacción, ver los datos y los tipos de datos que contiene

In [109]:
new_transaction = generateTransaction(10000, 1000, "2015-1-1 7:00:00", "2020-1-1 17:00:00")
new_transaction.show()
new_transaction.printSchema()

+------+-----------+-------------------+
|amount|customer_id|       purchased_at|
+------+-----------+-------------------+
|  7376|        352|2016-06-10 15:54:55|
+------+-----------+-------------------+

root
 |-- amount: long (nullable = true)
 |-- customer_id: long (nullable = true)
 |-- purchased_at: timestamp (nullable = true)



Se agrega la información generada a la base de datos

In [103]:
new_transaction.write \
    .format("jdbc") \
    .option("url", "jdbc:postgresql://localhost:5432/transactions") \
    .option("user", "postgres") \
    .option("password", "password") \
    .option("dbtable", "transactions") \
    .mode('append') \
    .save()